In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import timm
import numpy as np
import cv2
import random
import time
import json
import pickle
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from collections import Counter
import warnings
import multiprocessing as mp
import threading
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
import queue
import os
warnings.filterwarnings('ignore')

# GPU OPTIMIZATION: Enable mixed precision training if available
try:
    from torch.cuda.amp import autocast, GradScaler
    MIXED_PRECISION_AVAILABLE = True
except ImportError:
    MIXED_PRECISION_AVAILABLE = False

# Fix Windows multiprocessing issues
def fix_windows_multiprocessing():
    if hasattr(mp, 'set_start_method'):
        try:
            mp.set_start_method('spawn', force=True)
        except RuntimeError:
            pass
    
    import torch.multiprocessing as tmp
    tmp.set_sharing_strategy('file_system')

def load_original_data():
    """Load the original 357 images"""
    print("LOADING ORIGINAL DATA")
    print("="*50)
    
    try:
        from buck.analysis.basics import ingest_images
        
        fpath = "G:\\Dropbox\\AI Projects\\buck\\images\\squared\\color\\*_NDA.png"
        print(f"   Trying primary path: {fpath}")
        
        images, ages = ingest_images(fpath)
        
        print(f"   SUCCESS: Loaded {len(images)} original images from: {fpath}")
        
        print("   Grouping ages: 5.5+ -> 5.5")
        ages_grouped = []
        for age in ages:
            if age >= 5.5:
                ages_grouped.append(5.5)
            else:
                ages_grouped.append(age)
        
        if len(ages_grouped) == 0:
            print("   ERROR: No ages remaining after grouping")
            raise ValueError("No ages remaining after grouping")
        
        print(f"   Original age distribution: {dict(Counter(ages))}")
        print(f"   Grouped age distribution: {dict(Counter(ages_grouped))}")
        
        return images, ages_grouped
        
    except ImportError as e:
        print(f"   ERROR: Cannot import buck.analysis.basics: {e}")
        raise
    except Exception as e:
        print(f"   ERROR loading data: {e}")
        raise

def create_train_val_test_split(images, ages, test_size=0.2, val_size=0.15, random_state=42):
    """Create train/validation/test split"""
    print("\nCREATING TRAIN/VAL/TEST SPLIT")
    print("="*50)
    
    if len(images) == 0 or len(ages) == 0:
        print(f"   ERROR: Empty input data - images: {len(images)}, ages: {len(ages)}")
        raise ValueError("Input data is empty")
    
    if len(images) != len(ages):
        print(f"   ERROR: Mismatched data lengths - images: {len(images)}, ages: {len(ages)}")
        raise ValueError("Images and ages must have the same length")
    
    if not isinstance(images, np.ndarray):
        images = np.array(images)
    if not isinstance(ages, np.ndarray):
        ages = np.array(ages)
    
    age_counts = Counter(ages)
    
    if len(age_counts) == 0:
        print(f"   ERROR: No age data found")
        raise ValueError("Age counter is empty")
    
    min_count = min(age_counts.values())
    can_stratify = min_count >= 2
    
    print(f"   Age distribution: {dict(age_counts)}")
    print(f"   Minimum class size: {min_count}")
    print(f"   Can use stratified split: {can_stratify}")
    
    if can_stratify:
        X_temp, X_test, y_temp, y_test = train_test_split(
            images, ages, test_size=test_size, random_state=random_state, stratify=ages
        )
        
        val_size_adjusted = val_size / (1 - test_size)
        X_train, X_val, y_train, y_val = train_test_split(
            X_temp, y_temp, test_size=val_size_adjusted, random_state=random_state, stratify=y_temp
        )
    else:
        print("   WARNING: Using random split (some classes too small for stratification)")
        X_temp, X_test, y_temp, y_test = train_test_split(
            images, ages, test_size=test_size, random_state=random_state, shuffle=True
        )
        
        val_size_adjusted = val_size / (1 - test_size)
        X_train, X_val, y_train, y_val = train_test_split(
            X_temp, y_temp, test_size=val_size_adjusted, random_state=random_state, shuffle=True
        )
    
    unique_ages = sorted(list(set(ages)))
    label_mapping = {age: i for i, age in enumerate(unique_ages)}
    reverse_mapping = {i: age for age, i in label_mapping.items()}
    
    print(f"   Train: {len(X_train)} samples")
    print(f"   Val: {len(X_val)} samples") 
    print(f"   Test: {len(X_test)} samples")
    print(f"   Label mapping: {label_mapping}")
    print(f"   Number of classes: {len(unique_ages)}")
    
    y_train_indices = np.array([label_mapping[age] for age in y_train])
    y_val_indices = np.array([label_mapping[age] for age in y_val])
    y_test_indices = np.array([label_mapping[age] for age in y_test])
    
    print(f"   Train distribution: {Counter(y_train_indices)}")
    print(f"   Val distribution: {Counter(y_val_indices)}")
    print(f"   Test distribution: {Counter(y_test_indices)}")
    
    return (X_train, y_train_indices, X_val, y_val_indices, X_test, y_test_indices, 
            label_mapping, reverse_mapping)

def augment_image(image):
    """Apply random augmentation to an image"""
    if image.dtype != np.uint8:
        image = image.astype(np.uint8)
    
    if random.random() < 0.5:
        angle = random.uniform(-15, 15)
        h, w = image.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        image = cv2.warpAffine(image, M, (w, h))
    
    if random.random() < 0.5:
        image = cv2.flip(image, 1)
    
    if random.random() < 0.5:
        alpha = random.uniform(0.8, 1.2)
        beta = random.randint(-20, 20)
        image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    
    if random.random() < 0.3:
        noise = np.random.normal(0, 5, image.shape).astype(np.int16)
        image_int16 = image.astype(np.int16)
        noisy_image = np.clip(image_int16 + noise, 0, 255)
        image = noisy_image.astype(np.uint8)
    
    return image

def balance_and_augment_data(X_train, y_train, augment_multiplier=40, num_classes=5):
    """Balance classes and augment training data"""
    print(f"\nBALANCING AND AUGMENTING DATA")
    print("="*50)
    print(f"   Target: {augment_multiplier}x augmentation per class")
    
    class_counts = Counter(y_train)
    print(f"   Original distribution: {dict(class_counts)}")
    
    max_count = max(class_counts.values())
    target_count = max_count * augment_multiplier
    print(f"   Target samples per class: {target_count}")
    
    X_augmented = []
    y_augmented = []
    
    for class_idx in range(num_classes):
        class_mask = y_train == class_idx
        class_images = X_train[class_mask]
        class_labels = y_train[class_mask]
        
        current_count = len(class_images)
        needed_count = target_count
        
        print(f"   Class {class_idx}: {current_count} -> {needed_count} samples")
        
        X_augmented.extend(class_images)
        y_augmented.extend(class_labels)
        
        augmented_needed = needed_count - current_count
        
        for i in range(augmented_needed):
            original_idx = random.randint(0, current_count - 1)
            original_image = class_images[original_idx].copy()
            
            augmented_image = augment_image(original_image)
            
            X_augmented.append(augmented_image)
            y_augmented.append(class_idx)
    
    X_augmented = np.array(X_augmented)
    y_augmented = np.array(y_augmented)
    
    print(f"   Augmentation complete: {len(X_augmented)} total samples")
    print(f"   Final distribution: {Counter(y_augmented)}")
    
    return X_augmented, y_augmented

class DeerDataset(Dataset):
    """Dataset for deer aging with preprocessing"""
    
    def __init__(self, X, y, transform=True):
        if isinstance(X, np.ndarray):
            self.X = torch.FloatTensor(X)
        else:
            self.X = torch.FloatTensor(np.array(X))
            
        if isinstance(y, np.ndarray):
            self.y = torch.LongTensor(y)
        else:
            self.y = torch.LongTensor(np.array(y))
        
        self.transform = transform
        self.mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        self.std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        image = self.X[idx].clone()
        label = self.y[idx].clone()
        
        if image.max() > 1.0:
            image = image / 255.0
        
        if len(image.shape) == 3 and image.shape[-1] == 3:
            image = image.permute(2, 0, 1)
        
        if image.shape[-2:] != (224, 224):
            image = image.unsqueeze(0)
            image = F.interpolate(image, size=(224, 224), mode='bilinear', align_corners=False)
            image = image.squeeze(0)
        
        image = (image - self.mean) / self.std
        
        return image, label

def get_optimal_batch_size(device):
    """Determine optimal batch size based on device"""
    if device.type == 'cuda':
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"   GPU Memory: {gpu_memory:.1f} GB")
        
        if gpu_memory >= 16:
            return 64
        elif gpu_memory >= 8:
            return 32
        elif gpu_memory >= 4:
            return 24
        else:
            return 16
    else:
        import psutil
        ram_gb = psutil.virtual_memory().total / 1e9
        print(f"   System RAM: {ram_gb:.1f} GB")
        
        if ram_gb >= 32:
            return 16
        elif ram_gb >= 16:
            return 12
        else:
            return 8

def get_optimal_num_workers():
    """Get optimal number of workers for data loading"""
    import platform
    num_cores = mp.cpu_count()
    
    if platform.system() == "Windows":
        return min(4, max(1, num_cores // 2))
    else:
        return min(8, max(1, int(num_cores * 0.75)))

class CompleteDeerAgeTrainer:
    """Complete deer age trainer with parallelization"""
    
    def __init__(self, num_classes=5, max_parallel_models=2):
        self.num_classes = num_classes
        self.max_parallel_models = max_parallel_models
        
        # Fix Windows multiprocessing
        fix_windows_multiprocessing()
        
        if torch.cuda.is_available():
            self.device = torch.device('cuda')
            print(f"COMPLETE DEER AGE TRAINER")
            print(f"   Device: {self.device} ({torch.cuda.get_device_name(0)})")
            print(f"   CUDA Version: {torch.version.cuda}")
            print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
            
            torch.cuda.empty_cache()
            torch.backends.cudnn.benchmark = True
            torch.backends.cudnn.deterministic = False
            
        else:
            self.device = torch.device('cpu')
            print(f"COMPLETE DEER AGE TRAINER")
            print(f"   Device: {self.device} (CUDA not available)")
        
        print(f"   Classes: {num_classes}")
        print(f"   Max Parallel Models: {max_parallel_models}")
        self.all_results = []
        
        if MIXED_PRECISION_AVAILABLE and self.device.type == 'cuda':
            self.scaler = GradScaler()
            self.use_mixed_precision = True
            print(f"   Mixed Precision: Enabled")
        else:
            self.scaler = None
            self.use_mixed_precision = False
            print(f"   Mixed Precision: Disabled")
    
    def get_all_architectures_with_fallback(self):
        """Get ALL architectures with graceful fallback for missing pretrained weights"""
        
        architectures = {
            # ResNeXt
            'ResNeXt-50': {'model_name': 'resnext50_32x4d', 'family': 'ResNeXt'},
            'ResNeXt-101': {'model_name': 'resnext101_32x8d', 'family': 'ResNeXt'},
            
            # Wide ResNet
            'Wide-ResNet-50': {'model_name': 'wide_resnet50_2', 'family': 'Wide-ResNet'},
            'Wide-ResNet-101': {'model_name': 'wide_resnet101_2', 'family': 'Wide-ResNet'},
            
            # MobileNet Family
            'MobileNetV2': {'model_name': 'mobilenetv2_100', 'family': 'MobileNet'},
            'MobileNetV3-Small': {'model_name': 'mobilenetv3_small_100', 'family': 'MobileNet'},
            'MobileNetV3-Large': {'model_name': 'mobilenetv3_large_100', 'family': 'MobileNet'},
            
            # RegNet Family
            'RegNetX-400MF': {'model_name': 'regnetx_400mf', 'family': 'RegNet', 'alternatives': ['regnetx_002', 'regnetx_004']},
            'RegNetX-800MF': {'model_name': 'regnetx_800mf', 'family': 'RegNet', 'alternatives': ['regnetx_004', 'regnetx_006']},
            'RegNetY-400MF': {'model_name': 'regnety_400mf', 'family': 'RegNet', 'alternatives': ['regnety_002', 'regnety_004']},
            'RegNetY-800MF': {'model_name': 'regnety_800mf', 'family': 'RegNet', 'alternatives': ['regnety_004', 'regnety_006']},
            
            # ConvNeXt Family
            'ConvNeXt-Tiny': {'model_name': 'convnext_tiny', 'family': 'ConvNeXt', 'alternatives': ['convnext_tiny_in22ft1k']},
            'ConvNeXt-Small': {'model_name': 'convnext_small', 'family': 'ConvNeXt', 'alternatives': ['convnext_small_in22ft1k']},
            'ConvNeXt-Base': {'model_name': 'convnext_base', 'family': 'ConvNeXt', 'alternatives': ['convnext_base_in22ft1k']},
            
            # Vision Transformer variants
            'Swin-Tiny': {'model_name': 'swin_tiny_patch4_window7_224', 'family': 'Swin', 'alternatives': ['swin_tiny_patch4_window7_224_in22k']},
            'Swin-Small': {'model_name': 'swin_small_patch4_window7_224', 'family': 'Swin', 'alternatives': ['swin_small_patch4_window7_224_in22k']},
            
            # VGG (classic)
            'VGG-16': {'model_name': 'vgg16', 'family': 'VGG'},
            'VGG-19': {'model_name': 'vgg19', 'family': 'VGG'},
            
            # Vision Transformers
            'DeiT-Tiny': {'model_name': 'deit_tiny_patch16_224', 'family': 'DeiT'},
            'DeiT-Small': {'model_name': 'deit_small_patch16_224', 'family': 'DeiT'},
            'DeiT-Base': {'model_name': 'deit_base_patch16_224', 'family': 'DeiT'},
            
            # Additional EfficientNet variants
            'EfficientNet-ES': {'model_name': 'efficientnet_es', 'family': 'EfficientNet'},
            'EfficientNet-EM': {'model_name': 'efficientnet_em', 'family': 'EfficientNet'},
            'EfficientNet-EL': {'model_name': 'efficientnet_el', 'family': 'EfficientNet'},
            
            # Additional strong architectures
            'ResNet-26': {'model_name': 'resnet26', 'family': 'ResNet'},
            'ResNet-26d': {'model_name': 'resnet26d', 'family': 'ResNet'},
            'SEResNet-50': {'model_name': 'seresnet50', 'family': 'SEResNet'},
            'SEResNeXt-50': {'model_name': 'seresnext50_32x4d', 'family': 'SEResNeXt'},
        }
        
        print(f"\nCOMPLETE ARCHITECTURE ARSENAL ({len(architectures)} models)")
        print("="*80)
        print("FALLBACK STRATEGY: Pretrained -> Alternative Names -> Random Init")
        print("ALL models will be tested regardless of pretrained weight availability")
        
        families = {}
        for arch_name, arch_info in architectures.items():
            family = arch_info['family']
            if family not in families:
                families[family] = []
            families[family].append(arch_name)
        
        for family, models in families.items():
            print(f"{family} ({len(models)} models): {', '.join(models)}")
        
        return architectures
    
    def create_model_with_fallback(self, arch_name, arch_info, freeze_strategy='none'):
        """Create model with graceful fallback for missing pretrained weights"""
        model_name = arch_info['model_name']
        alternatives = arch_info.get('alternatives', [])
        
        print(f"      Creating {arch_name}...")
        
        try:
            print(f"         Trying pretrained: {model_name}")
            model = timm.create_model(model_name, pretrained=True, num_classes=self.num_classes)
            initialization_type = "pretrained"
            final_model_name = model_name
            print(f"         SUCCESS with pretrained weights!")
        except Exception as e1:
            print(f"         Pretrained failed: {str(e1)[:50]}...")
            
            model = None
            for alt_name in alternatives:
                try:
                    print(f"         Trying alternative pretrained: {alt_name}")
                    model = timm.create_model(alt_name, pretrained=True, num_classes=self.num_classes)
                    initialization_type = "pretrained_alt"
                    final_model_name = alt_name
                    print(f"         SUCCESS with alternative pretrained weights!")
                    break
                except Exception as e2:
                    print(f"         Alternative {alt_name} failed: {str(e2)[:30]}...")
                    continue
            
            if model is None:
                try:
                    print(f"         Falling back to random initialization: {model_name}")
                    model = timm.create_model(model_name, pretrained=False, num_classes=self.num_classes)
                    initialization_type = "random"
                    final_model_name = model_name
                    print(f"         SUCCESS with random initialization!")
                except Exception as e3:
                    for alt_name in alternatives:
                        try:
                            print(f"         Trying alternative random: {alt_name}")
                            model = timm.create_model(alt_name, pretrained=False, num_classes=self.num_classes)
                            initialization_type = "random_alt"
                            final_model_name = alt_name
                            print(f"         SUCCESS with alternative random initialization!")
                            break
                        except Exception as e4:
                            continue
                    
                    if model is None:
                        print(f"         COMPLETE FAILURE: All strategies failed")
                        return None, None, None
        
        if freeze_strategy == 'backbone':
            print(f"         Freezing backbone layers...")
            for name, param in model.named_parameters():
                if 'classifier' not in name and 'head' not in name and 'fc' not in name:
                    param.requires_grad = False
            
            trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
            total_params = sum(p.numel() for p in model.parameters())
            print(f"         Loaded: {total_params:,} total, {trainable_params:,} trainable ({initialization_type})")
        
        elif freeze_strategy == 'partial':
            print(f"         Partial freeze (last 30% unfrozen)...")
            all_params = list(model.named_parameters())
            total_layers = len(all_params)
            freeze_until = int(total_layers * 0.7)
            
            for i, (name, param) in enumerate(all_params):
                if i < freeze_until and 'classifier' not in name and 'head' not in name and 'fc' not in name:
                    param.requires_grad = False
            
            trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
            total_params = sum(p.numel() for p in model.parameters())
            print(f"         Loaded: {total_params:,} total, {trainable_params:,} trainable ({initialization_type})")
        
        else:
            total_params = sum(p.numel() for p in model.parameters())
            print(f"         Loaded: {total_params:,} parameters (all trainable, {initialization_type})")
        
        model = model.to(self.device)
        
        if hasattr(torch, 'compile') and self.device.type == 'cuda':
            try:
                model = torch.compile(model)
                print(f"         Model compiled for optimization")
            except Exception as e:
                print(f"         Model compilation failed (continuing without): {str(e)[:30]}")
        
        return model, initialization_type, final_model_name
    
    def ultra_aggressive_training(self, model, arch_name, train_loader, val_loader, test_loader, strategy='unfrozen'):
        """Ultra aggressive training with enhanced error handling"""
        print(f"      ULTRA AGGRESSIVE TRAINING: {arch_name} ({strategy})...")
        
        try:
            criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
            
            if strategy == 'frozen':
                lr = 0.01
                max_epochs = 50 if self.device.type == 'cpu' else 100
                patience = 25 if self.device.type == 'cpu' else 50
            elif strategy == 'partial':
                lr = 0.005
                max_epochs = 50 if self.device.type == 'cpu' else 100
                patience = 25 if self.device.type == 'cpu' else 50
            else:
                lr = 0.001
                max_epochs = 50 if self.device.type == 'cpu' else 100
                patience = 25 if self.device.type == 'cpu' else 50
            
            optimizer = optim.AdamW(
                model.parameters(),
                lr=lr,
                weight_decay=0.01,
                betas=(0.9, 0.999)
            )
            
            scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
            
            best_val_acc = 0.0
            patience_counter = 0
            
            print(f"         ULTRA SETUP: {max_epochs} epochs, LR={lr}, patience={patience}")
            if self.use_mixed_precision and self.device.type == 'cuda':
                print(f"         Mixed Precision: Enabled")
            else:
                print(f"         Mixed Precision: Disabled")
            
            successful_epochs = 0
            
            for epoch in range(max_epochs):
                try:
                    model.train()
                    train_correct = 0
                    train_total = 0
                    train_loss = 0.0
                    
                    batch_count = 0
                    for batch_idx, (images, labels) in enumerate(train_loader):
                        try:
                            if self.device.type == 'cuda':
                                images = images.to(self.device, non_blocking=True)
                                labels = labels.to(self.device, non_blocking=True)
                            else:
                                images = images.to(self.device)
                                labels = labels.to(self.device)
                            
                            optimizer.zero_grad()
                            
                            if self.use_mixed_precision and self.device.type == 'cuda':
                                with autocast():
                                    outputs = model(images)
                                    loss = criterion(outputs, labels)
                                
                                self.scaler.scale(loss).backward()
                                self.scaler.unscale_(optimizer)
                                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                                self.scaler.step(optimizer)
                                self.scaler.update()
                            else:
                                outputs = model(images)
                                loss = criterion(outputs, labels)
                                loss.backward()
                                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                                optimizer.step()
                            
                            train_loss += loss.item()
                            _, predicted = torch.max(outputs.data, 1)
                            train_total += labels.size(0)
                            train_correct += (predicted == labels).sum().item()
                            
                            batch_count += 1
                            
                            if self.device.type == 'cpu' and batch_count % 50 == 0:
                                current_acc = 100 * train_correct / train_total if train_total > 0 else 0
                                print(f"           Batch {batch_count}/{len(train_loader)}: {current_acc:.1f}%", end='\r')
                        
                        except Exception as batch_error:
                            print(f"         Batch {batch_idx} failed: {str(batch_error)}")
                            continue
                    
                    if train_total == 0:
                        print(f"         Epoch {epoch}: No training data processed - skipping")
                        continue
                    
                    train_acc = 100 * train_correct / train_total
                    
                    model.eval()
                    val_correct = 0
                    val_total = 0
                    val_loss = 0.0
                    
                    with torch.no_grad():
                        for images, labels in val_loader:
                            try:
                                if self.device.type == 'cuda':
                                    images = images.to(self.device, non_blocking=True)
                                    labels = labels.to(self.device, non_blocking=True)
                                else:
                                    images = images.to(self.device)
                                    labels = labels.to(self.device)
                                
                                if self.use_mixed_precision and self.device.type == 'cuda':
                                    with autocast():
                                        outputs = model(images)
                                        loss = criterion(outputs, labels)
                                else:
                                    outputs = model(images)
                                    loss = criterion(outputs, labels)
                                
                                val_loss += loss.item()
                                _, predicted = torch.max(outputs.data, 1)
                                val_total += labels.size(0)
                                val_correct += (predicted == labels).sum().item()
                            
                            except Exception as val_batch_error:
                                print(f"         Validation batch failed: {str(val_batch_error)}")
                                continue
                    
                    if val_total == 0:
                        print(f"         Epoch {epoch}: No validation data processed - skipping")
                        continue
                    
                    val_acc = 100 * val_correct / val_total
                    scheduler.step()
                    current_lr = scheduler.get_last_lr()[0]
                    
                    successful_epochs += 1
                    
                    if val_acc > best_val_acc:
                        best_val_acc = val_acc
                        patience_counter = 0
                        best_model_state = model.state_dict().copy()
                        improvement = "BEST"
                    else:
                        patience_counter += 1
                        improvement = ""
                    
                    if epoch % 5 == 0 or epoch < 10 or improvement or epoch > max_epochs - 10:
                        gap = train_acc - val_acc
                        print(f"         Epoch {epoch:3d}: Train {train_acc:.1f}%, Val {val_acc:.1f}% (gap: {gap:+.1f}%), LR: {current_lr:.2e} {improvement}")
                    
                    if patience_counter >= patience:
                        print(f"         Early stopping at epoch {epoch} (patience={patience})")
                        break
                        
                except Exception as epoch_error:
                    print(f"         Epoch {epoch} failed completely: {str(epoch_error)}")
                    continue
            
            if successful_epochs == 0:
                print(f"         TRAINING FAILED: No epochs completed successfully")
                return 0.0, 0.0
            
            if 'best_model_state' in locals():
                model.load_state_dict(best_model_state)
            
            model.eval()
            test_correct = 0
            test_total = 0
            
            with torch.no_grad():
                for images, labels in test_loader:
                    try:
                        if self.device.type == 'cuda':
                            images = images.to(self.device, non_blocking=True)
                            labels = labels.to(self.device, non_blocking=True)
                        else:
                            images = images.to(self.device)
                            labels = labels.to(self.device)
                        
                        if self.use_mixed_precision and self.device.type == 'cuda':
                            with autocast():
                                outputs = model(images)
                        else:
                            outputs = model(images)
                        
                        _, predicted = torch.max(outputs.data, 1)
                        test_total += labels.size(0)
                        test_correct += (predicted == labels).sum().item()
                    
                    except Exception as test_batch_error:
                        print(f"         Test batch failed: {str(test_batch_error)}")
                        continue
            
            if test_total == 0:
                print(f"         TEST FAILED: No test data processed")
                return best_val_acc, 0.0
            
            test_acc = 100 * test_correct / test_total
            
            print(f"         {arch_name} ({strategy}) FINAL: Val {best_val_acc:.1f}%, Test {test_acc:.1f}%")
            
            return best_val_acc, test_acc
            
        except Exception as training_error:
            print(f"         TRAINING COMPLETELY FAILED: {str(training_error)}")
            return 0.0, 0.0
        
        finally:
            if self.device.type == 'cuda':
                torch.cuda.empty_cache()
    
    def train_architecture_worker(self, arch_info, train_loader, val_loader, test_loader):
        """Worker function to train a single architecture"""
        arch_name, arch_details = arch_info
        try:
            results = self.test_architecture_with_multiple_strategies(
                arch_name, arch_details, train_loader, val_loader, test_loader
            )
            return (arch_name, results)
        except Exception as e:
            print(f"         Architecture {arch_name} failed: {str(e)}")
            return (arch_name, [])
    
    def test_architecture_with_multiple_strategies(self, arch_name, arch_info, train_loader, val_loader, test_loader):
        """Test architecture with multiple training strategies"""
        results = []
        
        print(f"      FROZEN BACKBONE STRATEGY:")
        model_frozen, init_type_frozen, final_name_frozen = self.create_model_with_fallback(arch_name, arch_info, freeze_strategy='backbone')
        if model_frozen is not None:
            try:
                val_acc_frozen, test_acc_frozen = self.ultra_aggressive_training(
                    model_frozen, arch_name, train_loader, val_loader, test_loader, strategy='frozen'
                )
                results.append({
                    'name': f"{arch_name}-Frozen",
                    'strategy': 'frozen',
                    'val_accuracy': val_acc_frozen,
                    'test_accuracy': test_acc_frozen,
                    'family': arch_info['family'],
                    'initialization': init_type_frozen,
                    'final_model_name': final_name_frozen,
                    'original_model_name': arch_info['model_name']
                })
            except Exception as e:
                print(f"         Frozen strategy failed: {str(e)[:50]}...")
            finally:
                del model_frozen
                if self.device.type == 'cuda':
                    torch.cuda.empty_cache()
        
        if results and results[-1]['val_accuracy'] > 35:
            print(f"      PARTIAL FREEZE STRATEGY:")
            model_partial, init_type_partial, final_name_partial = self.create_model_with_fallback(arch_name, arch_info, freeze_strategy='partial')
            if model_partial is not None:
                try:
                    val_acc_partial, test_acc_partial = self.ultra_aggressive_training(
                        model_partial, arch_name, train_loader, val_loader, test_loader, strategy='partial'
                    )
                    results.append({
                        'name': f"{arch_name}-Partial",
                        'strategy': 'partial',
                        'val_accuracy': val_acc_partial,
                        'test_accuracy': test_acc_partial,
                        'family': arch_info['family'],
                        'initialization': init_type_partial,
                        'final_model_name': final_name_partial,
                        'original_model_name': arch_info['model_name']
                    })
                except Exception as e:
                    print(f"         Partial strategy failed: {str(e)[:50]}...")
                finally:
                    del model_partial
                    if self.device.type == 'cuda':
                        torch.cuda.empty_cache()
        
        if results and max(r['val_accuracy'] for r in results) > 45:
            print(f"      FULL UNFROZEN STRATEGY:")
            model_unfrozen, init_type_unfrozen, final_name_unfrozen = self.create_model_with_fallback(arch_name, arch_info, freeze_strategy='none')
            if model_unfrozen is not None:
                try:
                    val_acc_unfrozen, test_acc_unfrozen = self.ultra_aggressive_training(
                        model_unfrozen, arch_name, train_loader, val_loader, test_loader, strategy='unfrozen'
                    )
                    results.append({
                        'name': f"{arch_name}-Unfrozen",
                        'strategy': 'unfrozen',
                        'val_accuracy': val_acc_unfrozen,
                        'test_accuracy': test_acc_unfrozen,
                        'family': arch_info['family'],
                        'initialization': init_type_unfrozen,
                        'final_model_name': final_name_unfrozen,
                        'original_model_name': arch_info['model_name']
                    })
                except Exception as e:
                    print(f"         Unfrozen strategy failed: {str(e)[:50]}...")
                finally:
                    del model_unfrozen
                    if self.device.type == 'cuda':
                        torch.cuda.empty_cache()
        
        return results
    
    def run_complete_pipeline(self, X_train, y_train, X_val, y_val, X_test, y_test):
        """Run the complete pipeline with parallelization"""
        print("COMPLETE DEER AGING PIPELINE")
        print("="*80)
        print("Starting from EfficientNet-B5 onwards")
        print("All results will be saved automatically")
        print("="*80)
        
        train_dataset = DeerDataset(X_train, y_train)
        val_dataset = DeerDataset(X_val, y_val)
        test_dataset = DeerDataset(X_test, y_test)
        
        if self.device.type == 'cpu':
            optimal_batch_size = 8
            optimal_num_workers = 0
        else:
            optimal_batch_size = get_optimal_batch_size(self.device)
            optimal_num_workers = get_optimal_num_workers()
        
        print(f"OPTIMIZATIONS:")
        print(f"   Optimal batch size: {optimal_batch_size}")
        print(f"   Num workers: {optimal_num_workers}")
        print(f"   Pin memory: {self.device.type == 'cuda'}")
        print(f"   Max parallel models: {self.max_parallel_models}")
        
        # Create data loaders with proper multiprocessing context
        loader_kwargs = {
            'batch_size': optimal_batch_size,
            'num_workers': optimal_num_workers,
            'pin_memory': (self.device.type == 'cuda'),
            'persistent_workers': (optimal_num_workers > 0),
        }
        
        if optimal_num_workers > 0:
            loader_kwargs['multiprocessing_context'] = mp.get_context('spawn')
            loader_kwargs['worker_init_fn'] = lambda worker_id: np.random.seed(np.random.get_state()[1][0] + worker_id)
        
        train_loader = DataLoader(train_dataset, shuffle=True, **loader_kwargs)
        val_loader = DataLoader(val_dataset, shuffle=False, **loader_kwargs)
        test_loader = DataLoader(test_dataset, shuffle=False, **loader_kwargs)
        
        print(f"Data ready: {len(train_dataset)} train, {len(val_dataset)} val, {len(test_dataset)} test")
        
        architectures = self.get_all_architectures_with_fallback()
        
        total_start_time = time.time()
        
        print(f"\nULTRA AGGRESSIVE TESTING: {len(architectures)} ARCHITECTURES")
        print("="*80)
        
        # Process architectures in parallel batches
        arch_items = list(architectures.items())
        
        if self.max_parallel_models > 1:
            print(f"PARALLEL PROCESSING: {self.max_parallel_models} models at once")
            
            for i in range(0, len(arch_items), self.max_parallel_models):
                batch = arch_items[i:i + self.max_parallel_models]
                batch_start_time = time.time()
                
                print(f"\nPARALLEL BATCH: {[item[0] for item in batch]}")
                
                with ThreadPoolExecutor(max_workers=self.max_parallel_models) as executor:
                    future_to_arch = {
                        executor.submit(self.train_architecture_worker, arch_item, 
                                       train_loader, val_loader, test_loader): arch_item[0]
                        for arch_item in batch
                    }
                    
                    for future in as_completed(future_to_arch):
                        arch_name = future_to_arch[future]
                        try:
                            arch_name_result, results = future.result()
                            
                            for result in results:
                                result['architecture_family'] = arch_name_result
                                result['training_time'] = time.time() - batch_start_time
                                result['timestamp'] = datetime.now().isoformat()
                                self.all_results.append(result)
                            
                            if results:
                                best_arch_result = max(results, key=lambda x: x['test_accuracy'])
                                print(f"      Best {arch_name_result}: {best_arch_result['name']} ({best_arch_result['test_accuracy']:.1f}%)")
                                
                        except Exception as e:
                            print(f"      Error collecting {arch_name}: {str(e)}")
                
                print(f"PARALLEL BATCH COMPLETED: {time.time() - batch_start_time:.1f}s")
                
                # Save progress
                elapsed_time = time.time() - total_start_time
                progress_file = self.save_progress_text_file(i//self.max_parallel_models + 1, 
                                                           len(arch_items)//self.max_parallel_models + 1, 
                                                           f"Batch-{i//self.max_parallel_models + 1}", 
                                                           [], elapsed_time)
                json_file, pkl_file = self.save_backup_files(i//self.max_parallel_models + 1)
                print(f"      Backups: {json_file}, {pkl_file}")
        
        else:
            # Sequential processing
            for i, (arch_name, arch_info) in enumerate(arch_items, 1):
                print(f"\n[{i}/{len(arch_items)}] ULTRA AGGRESSIVE {arch_name}")
                print("-" * 70)
                
                start_time = time.time()
                
                arch_results = self.test_architecture_with_multiple_strategies(
                    arch_name, arch_info, train_loader, val_loader, test_loader
                )
                
                for result in arch_results:
                    result['architecture_family'] = arch_info['family']
                    result['training_time'] = time.time() - start_time
                    result['timestamp'] = datetime.now().isoformat()
                    self.all_results.append(result)
                
                if arch_results:
                    best_arch_result = max(arch_results, key=lambda x: x['test_accuracy'])
                    print(f"      Best {arch_name}: {best_arch_result['name']} ({best_arch_result['test_accuracy']:.1f}%)")
                
                print(f"      Total time for {arch_name}: {time.time() - start_time:.1f}s")
                
                elapsed_time = time.time() - total_start_time
                progress_file = self.save_progress_text_file(i, len(arch_items), arch_name, arch_results, elapsed_time)
                json_file, pkl_file = self.save_backup_files(i)
                
                print(f"      Backups: {json_file}, {pkl_file}")
                
                if i % 3 == 0:
                    self.show_intermediate_leaderboard(i)
        
        total_time = time.time() - total_start_time
        
        self.save_final_results(total_time)
        self.show_final_leaderboard(total_time)
        
        return self.all_results
    
    def save_progress_text_file(self, completed_count, total_count, arch_name, arch_results, total_time_so_far):
        """Save human-readable progress text file after each architecture"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f'deer_aging_progress_{timestamp}.txt'
        
        with open(filename, 'w', encoding='utf-8') as f:
            f.write("DEER AGING PIPELINE - PROGRESS REPORT\n")
            f.write("="*80 + "\n")
            f.write(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Runtime so far: {total_time_so_far/3600:.2f} hours\n")
            f.write(f"Progress: {completed_count}/{total_count} architectures completed\n")
            f.write(f"Device: {self.device}\n")
            f.write(f"Classes: {self.num_classes}\n")
            f.write(f"Mixed Precision: {self.use_mixed_precision}\n")
            f.write(f"Max Parallel Models: {self.max_parallel_models}\n")
            f.write("="*80 + "\n\n")
            
            if arch_results:
                f.write(f"JUST COMPLETED: {arch_name}\n")
                f.write("-"*50 + "\n")
                for result in arch_results:
                    init_type = result.get('initialization', 'unknown')
                    f.write(f"   {result['name']:30} | {result['strategy']:8} | {init_type:12} | Val: {result['val_accuracy']:5.1f}% | Test: {result['test_accuracy']:5.1f}%\n")
                
                best_arch = max(arch_results, key=lambda x: x['test_accuracy'])
                f.write(f"   Best: {best_arch['name']} ({best_arch['test_accuracy']:.1f}%)\n")
            f.write("\n")
            
            sorted_results = sorted(self.all_results, key=lambda x: x['test_accuracy'], reverse=True)
            f.write(f"CURRENT TOP 10 LEADERBOARD\n")
            f.write("-"*80 + "\n")
            f.write(f"{'Rank':<4} {'Model':<30} {'Strategy':<10} {'Init':<12} {'Val%':<8} {'Test%':<8}\n")
            f.write("-"*80 + "\n")
            
            for i, result in enumerate(sorted_results[:10], 1):
                init_type = result.get('initialization', 'unknown')
                if init_type == 'pretrained':
                    init_display = "Pretrained"
                elif init_type == 'pretrained_alt':
                    init_display = "Alt-Pre"
                elif init_type == 'random':
                    init_display = "Random"
                elif init_type == 'random_alt':
                    init_display = "Alt-Rand"
                else:
                    init_display = "Unknown"
                
                f.write(f"{i:<4} {result['name']:<30} {result['strategy']:<10} {init_display:<12} {result['val_accuracy']:<7.1f} {result['test_accuracy']:<7.1f}\n")
            
            f.write("\n")
            
            if sorted_results:
                best_overall = sorted_results[0]
                breakthrough_count = sum(1 for r in sorted_results if r['test_accuracy'] > 54.2)
                excellent_count = sum(1 for r in sorted_results if r['test_accuracy'] >= 65.0)
                
                f.write(f"SUMMARY STATISTICS\n")
                f.write("-"*50 + "\n")
                f.write(f"Current Best: {best_overall['name']} ({best_overall['test_accuracy']:.1f}%)\n")
                f.write(f"Models beating 54.2% baseline: {breakthrough_count}/{len(sorted_results)}\n")
                f.write(f"Models achieving 65%+: {excellent_count}\n")
                f.write(f"Total models tested: {len(sorted_results)}\n")
                
                f.write(f"\nBY INITIALIZATION TYPE:\n")
                init_groups = {}
                for result in sorted_results:
                    init_type = result.get('initialization', 'unknown')
                    if init_type not in init_groups:
                        init_groups[init_type] = []
                    init_groups[init_type].append(result)
                
                for init_type, group in init_groups.items():
                    avg_test = sum(r['test_accuracy'] for r in group) / len(group)
                    best_test = max(r['test_accuracy'] for r in group)
                    f.write(f"   {init_type:15}: {len(group):2d} models, avg: {avg_test:.1f}%, best: {best_test:.1f}%\n")
            
            f.write("\n")
            f.write(f"REMAINING: {total_count - completed_count} architectures to test\n")
            estimated_time_remaining = (total_time_so_far / completed_count) * (total_count - completed_count) if completed_count > 0 else 0
            f.write(f"Estimated time remaining: {estimated_time_remaining/3600:.1f} hours\n")
            f.write("="*80 + "\n")
            f.write(f"This file: {filename}\n")
            f.write(f"Auto-saved at: {datetime.now().strftime('%H:%M:%S')}\n")
        
        print(f"      Progress saved: {filename}")
        return filename
    
    def save_backup_files(self, completed_count):
        """Save JSON/pickle backup files after each architecture"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        json_file = f'deer_aging_backup_{completed_count}_{timestamp}.json'
        with open(json_file, 'w') as f:
            json.dump(self.all_results, f, indent=2)
        
        pkl_file = f'deer_aging_backup_{completed_count}_{timestamp}.pkl'
        with open(pkl_file, 'wb') as f:
            pickle.dump(self.all_results, f)
        
        return json_file, pkl_file
    
    def save_final_results(self, total_time):
        """Save comprehensive final results"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        final_data = {
            'experiment_info': {
                'timestamp': timestamp,
                'total_runtime_hours': total_time / 3600,
                'total_models_tested': len(self.all_results),
                'device': str(self.device),
                'num_classes': self.num_classes,
                'mixed_precision': self.use_mixed_precision,
                'max_parallel_models': self.max_parallel_models
            },
            'results': self.all_results,
            'leaderboard': sorted(self.all_results, key=lambda x: x['test_accuracy'], reverse=True)
        }
        
        with open(f'deer_aging_final_results_{timestamp}.json', 'w') as f:
            json.dump(final_data, f, indent=2)
        
        with open(f'deer_aging_final_results_{timestamp}.pkl', 'wb') as f:
            pickle.dump(final_data, f)
        
        print(f"\nRESULTS SAVED:")
        print(f"   deer_aging_final_results_{timestamp}.json")
        print(f"   deer_aging_final_results_{timestamp}.pkl")
    
    def show_intermediate_leaderboard(self, completed_count):
        """Show intermediate leaderboard"""
        current_best = sorted(self.all_results, key=lambda x: x['test_accuracy'], reverse=True)[:5]
        print(f"\nCURRENT TOP 5 (after {completed_count} architectures):")
        for j, result in enumerate(current_best, 1):
            print(f"   {j}. {result['name']}: {result['test_accuracy']:.1f}%")
        print()
    
    def show_final_leaderboard(self, total_time):
        """Show comprehensive final leaderboard"""
        sorted_results = sorted(self.all_results, key=lambda x: x['test_accuracy'], reverse=True)
        
        print(f"\nFINAL COMPREHENSIVE RESULTS")
        print("="*80)
        print(f"Total testing time: {total_time/3600:.1f} hours")
        print(f"Models tested: {len(self.all_results)}")
        print(f"Parallelization: {self.max_parallel_models} concurrent models")
        print("="*80)
        print(f"{'Rank':<4} {'Model':<30} {'Strategy':<10} {'Init':<12} {'Val%':<8} {'Test%':<8} {'Status'}")
        print("-" * 80)
        
        for i, result in enumerate(sorted_results, 1):
            val_acc = result['val_accuracy']
            test_acc = result['test_accuracy']
            strategy = result['strategy']
            init_type = result.get('initialization', 'unknown')
            
            if init_type == 'pretrained':
                init_display = "Pretrained"
            elif init_type == 'pretrained_alt':
                init_display = "Alt-Pre"
            elif init_type == 'random':
                init_display = "Random"
            elif init_type == 'random_alt':
                init_display = "Alt-Rand"
            else:
                init_display = "Unknown"
            
            if test_acc >= 75.0:
                status = "BREAKTHROUGH!"
            elif test_acc >= 65.0:
                status = "EXCELLENT!"
            elif test_acc > 54.2:
                status = "NEW BEST!"
            elif test_acc > 45.0:
                status = "Good"
            else:
                status = "Weak"
            
            print(f"{i:<4} {result['name']:<30} {strategy:<10} {init_display:<12} {val_acc:<7.1f} {test_acc:<7.1f} {status}")
        
        print(f"\nANALYSIS BY INITIALIZATION:")
        init_groups = {}
        for result in sorted_results:
            init_type = result.get('initialization', 'unknown')
            if init_type not in init_groups:
                init_groups[init_type] = []
            init_groups[init_type].append(result)
        
        for init_type, group in init_groups.items():
            avg_test = sum(r['test_accuracy'] for r in group) / len(group)
            best_test = max(r['test_accuracy'] for r in group)
            print(f"   {init_type:15}: {len(group):2d} models, avg: {avg_test:.1f}%, best: {best_test:.1f}%")
        
        if sorted_results:
            best = sorted_results[0]
            breakthrough_count = sum(1 for r in sorted_results if r['test_accuracy'] > 54.2)
            excellent_count = sum(1 for r in sorted_results if r['test_accuracy'] >= 65.0)
            
            print(f"\nFINAL SUMMARY:")
            print(f"   ULTIMATE CHAMPION: {best['name']} ({best['test_accuracy']:.1f}%)")
            print(f"   Beat 54.2% baseline: {breakthrough_count}/{len(sorted_results)} models")
            print(f"   Achieved 65%+: {excellent_count} models")
            
            if best['test_accuracy'] >= 75.0:
                print(f"   MISSION ACCOMPLISHED! Achieved 75%+ accuracy!")
            elif best['test_accuracy'] >= 65.0:
                print(f"   EXCELLENT! Found 65%+ architecture!")
            elif best['test_accuracy'] > 54.2:
                improvement = best['test_accuracy'] - 54.2
                print(f"   SUCCESS! Improved by +{improvement:.1f}% over baseline!")
            
            best_pretrained = max([r for r in sorted_results if r.get('initialization', '').startswith('pretrained')], 
                                key=lambda x: x['test_accuracy'], default=None)
            best_random = max([r for r in sorted_results if r.get('initialization', '').startswith('random')], 
                            key=lambda x: x['test_accuracy'], default=None)
            
            if best_pretrained and best_random:
                print(f"   Best Pretrained: {best_pretrained['name']} ({best_pretrained['test_accuracy']:.1f}%)")
                print(f"   Best Random Init: {best_random['name']} ({best_random['test_accuracy']:.1f}%)")
                if best_random['test_accuracy'] > best_pretrained['test_accuracy']:
                    print(f"   SURPRISE! Random initialization outperformed pretrained!")
        
        print("="*80)

def run_complete_deer_aging_pipeline():
    """Run the complete deer aging pipeline from start to finish"""
    print("LAUNCHING COMPLETE DEER AGING PIPELINE")
    print("="*80)
    print("PIPELINE STEPS:")
    print("   1. Load original 357 images")
    print("   2. Create train/val/test splits")
    print("   3. Balance and augment training data")
    print("   4. Test all architectures (starting from EfficientNet-B5)")
    print("   5. Save results and create leaderboard")
    print("="*80)
    print("CRASH RECOVERY: Progress saved after each model!")
    print("Look for 'deer_aging_progress_*.txt' files for latest results")
    print("JSON/pickle backups: 'deer_aging_backup_*.json/.pkl'")
    print("="*80)
    
    try:
        images, ages = load_original_data()
        
        X_train, y_train, X_val, y_val, X_test, y_test, label_mapping, reverse_mapping = create_train_val_test_split(images, ages)
        
        X_train_aug, y_train_aug = balance_and_augment_data(X_train, y_train, augment_multiplier=40, num_classes=len(label_mapping))
        
        trainer = CompleteDeerAgeTrainer(num_classes=len(label_mapping), max_parallel_models=2)
        results = trainer.run_complete_pipeline(X_train_aug, y_train_aug, X_val, y_val, X_test, y_test)
        
        print("\nPIPELINE COMPLETE!")
        print("All results saved with timestamps")
        print("Check the final leaderboard above")
        
        return results, label_mapping, reverse_mapping
        
    except KeyboardInterrupt:
        print("\nINTERRUPTED BY USER")
        print("Check latest 'deer_aging_progress_*.txt' file for current results")
        print("Backup files saved as 'deer_aging_backup_*.json/.pkl'")
        raise
    except Exception as e:
        print(f"\nPIPELINE CRASHED: {str(e)}")
        print("Check latest 'deer_aging_progress_*.txt' file for results up to crash")
        print("Backup files saved as 'deer_aging_backup_*.json/.pkl'")
        print("You can manually load the pickle files to recover results")
        raise

def test_data_loading():
    """Test function to debug data loading issues"""
    print("TESTING DATA LOADING")
    print("="*50)
    
    import glob
    import os
    
    test_paths = [
        "G:\\Dropbox\\AI Projects\\buck\\images\\squared\\color\\*_NDA.png",
    ]
    
    print(f"Current working directory: {os.getcwd()}")
    
    for fpath in test_paths:
        print(f"\nTesting path: {fpath}")
        files = glob.glob(fpath)
        print(f"   Files found: {len(files)}")
        
        if len(files) > 0:
            print(f"   First few files: {files[:3]}")
            print(f"   This path works!")
            break
        else:
            print("   No files found at this path")
      
    try:
        from buck.analysis.basics import ingest_images
        print("\nSuccessfully imported buck.analysis.basics")
    except ImportError as e:
        print(f"\nCannot import buck.analysis.basics: {e}")
        print("Possible solutions:")
        print("   1. Make sure you're in the right directory")
        print("   2. Add buck to Python path: sys.path.append('/path/to/buck')")
        print("   3. Install buck package if it's a separate package")
        return False
    
    for fpath in test_paths:
        files = glob.glob(fpath)
        if len(files) > 0:
            try:
                print(f"\nTesting ingest_images with: {fpath}")
                images, ages = ingest_images(fpath)
                print(f"ingest_images returned: {len(images)} images, {len(ages)} ages")
                if len(ages) > 0:
                    print(f"Sample ages: {ages[:5]}")
                    print(f"Age range: {min(ages)} to {max(ages)}")
                    print(f"Unique ages: {sorted(set(ages))}")
                return True
            except Exception as e:
                print(f"ingest_images failed: {e}")
                continue
    
    return False

def load_latest_results():
    """Load the most recent backup results (for crash recovery)"""
    import glob
    import os
    
    print("CRASH RECOVERY MODE")
    print("="*50)
    
    pickle_files = glob.glob('deer_aging_backup_*.pkl')
    if not pickle_files:
        print("No backup files found!")
        return None
    
    latest_pickle = max(pickle_files, key=os.path.getctime)
    print(f"Loading latest backup: {latest_pickle}")
    
    with open(latest_pickle, 'rb') as f:
        results = pickle.load(f)
    
    print(f"Loaded {len(results)} results")
    
    if results:
        sorted_results = sorted(results, key=lambda x: x['test_accuracy'], reverse=True)
        print(f"Best model: {sorted_results[0]['name']} ({sorted_results[0]['test_accuracy']:.1f}%)")
        print(f"Models tested: {len(results)}")
        
        progress_files = glob.glob('deer_aging_progress_*.txt')
        if progress_files:
            latest_progress = max(progress_files, key=os.path.getctime)
            print(f"Latest progress report: {latest_progress}")
    
    return results

if __name__ == "__main__":
    mp.freeze_support()
    
    print("LAUNCHING COMPLETE DEER AGING PIPELINE...")
    print("="*80)
    
    print("Testing data loading before starting pipeline...")
    if not test_data_loading():
        print("\nDATA LOADING FAILED!")
        print("Please fix the data loading issue before running the pipeline")
        print("Run test_data_loading() to debug the issue")
    else:
        print("Data loading test passed!")
        print("\nStarting main pipeline...")
        final_results, final_label_mapping, final_reverse_mapping = run_complete_deer_aging_pipeline()

LAUNCHING COMPLETE DEER AGING PIPELINE...
Testing data loading before starting pipeline...
TESTING DATA LOADING
Current working directory: G:\Dropbox\AI Projects\buck\examples

Testing path: G:\Dropbox\AI Projects\buck\images\squared\color\*_NDA.png
ERROR! Session/line number was not unique in database. History logging moved to new session 89
   Files found: 197
   First few files: ['G:\\Dropbox\\AI Projects\\buck\\images\\squared\\color\\111009_111009_KY_2p5_NDA.png', 'G:\\Dropbox\\AI Projects\\buck\\images\\squared\\color\\171220_171220_MO_2p5_NDA.png', 'G:\\Dropbox\\AI Projects\\buck\\images\\squared\\color\\171220_171220_MO_3p5_NDA.png']
   This path works!

Successfully imported buck.analysis.basics

Testing ingest_images with: G:\Dropbox\AI Projects\buck\images\squared\color\*_NDA.png
ingest_images returned: 196 images, 196 ages
Sample ages: [2.5, 2.5, 3.5, 2.5, 3.5]
Age range: 1.5 to 12.5
Unique ages: [1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 8.5, 12.5]
Data loading test passed!

Starting 